In [1]:
import torch 
import matplotlib.pyplot as plt

## MLP class

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Neuron(nn.Module):
    def __init__(self, inputs: int, activation=None):
        super(Neuron, self).__init__()
        self.linear = nn.Linear(inputs, 1) 
        self.activation = activation if activation is not None else lambda x: x
    
    
    def forward(self, x):
        return self.activation(self.linear(x))

class Layer(nn.Module):
    def __init__(self, inputs: int, n_neurons: int, activation=None):
        super(Layer, self).__init__()
        self.linear = nn.Linear(inputs, n_neurons)
        self.activation = activation if activation is not None else lambda x: x
    
    def forward(self, x):
        x = self.linear(x)
        if self.activation is not None:
            x = self.activation(x)
        return x

class MLP(nn.Module):
    def __init__(self, layer_sizes, activation_funcs=None):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(layer_sizes) - 1):
            activation = activation_funcs[i] if activation_funcs and i < len(activation_funcs) else None
            layers.append(Layer(layer_sizes[i], layer_sizes[i+1], activation))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


## MLP class testing using MNIST dataset

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [4]:
#Data loading and normalizing
transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Normalize((0.5,), (0.5,))  
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


100%|██████████| 9912422/9912422 [00:01<00:00, 7681419.03it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5511065.65it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4541246.43it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [5]:
#Model initializing
model = MLP(layer_sizes=[784, 128, 64, 10], activation_funcs=[F.relu, F.relu])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [6]:
#Training loop
epochs = 5
for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        # Flatten MNIST images into a 784 vector
        images = images.view(images.shape[0], -1)
        
        optimizer.zero_grad() 
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Training loss: {running_loss/len(train_loader)}")

Epoch 1 - Training loss: 0.3367726824411165
Epoch 2 - Training loss: 0.17689169780103
Epoch 3 - Training loss: 0.14348050438886337
Epoch 4 - Training loss: 0.12617953160732015
Epoch 5 - Training loss: 0.11712130051213446
